### 环境

In [148]:
import os,sys
import time
import ipaddress
import numpy as np
import capnp

ws_root = r"E:\bitbucket\rhea_ws"
sys.path += [ws_root]
from common.utils_py import *

In [149]:
from ws_world import World

world = World(ws_root)
world.mountDevices()
world.startGrab()

In [150]:
world.getPose()

Pose.fromP6(231.51, 479.99, 34.80,  1.5660, 0.5469, 3.1088)

In [151]:
world.getJoints()

array([ 0.76985608, -1.05134451,  2.01196178, -0.96630774, -2.34146154,
       -0.54298459])

In [ ]:
##rxxxxxxxxxx
d = 20
d1 = (d/180.0*np.pi)
d2 = -(2*d/180.0*np.pi)
t = [0.0, 0.01, 0.1540]

""" world.movePoseDelta(rx=d1, line=True, tool=t)
time.sleep(0.5)
world.movePoseDelta(rx=d2, line=True, tool=t)
time.sleep(0.5)
world.movePoseDelta(rx=d1, line=True, tool=t) """


##ryyyyyyyyyyyy
""" world.movePoseDelta(ry=d1, line=True, tool=t)
time.sleep(0.5)
world.movePoseDelta(ry=d2, line=True, tool=t)
time.sleep(0.5)
world.movePoseDelta(ry=d1, line=True, tool=t) """


##rzzzzzzzzzz
""" world.movePoseDelta(rz=d1, line=True, tool=t)
time.sleep(0.5)
world.movePoseDelta(rz=d2, line=True, tool=t)
time.sleep(0.5)
world.movePoseDelta(rz=d1, line=True, tool=t) """

# 水平相机

In [152]:
def extend_pose(corner1, corner2, points, three_axis, rotation, tool):
    mesh_pose = []
    mesh_idx_pair = []
    for idx, d in enumerate(corner1 - corner2):
        if abs(d) > 0.1:
            mesh_idx_pair.append((idx, d))

    assert len(mesh_idx_pair) == 2, f"必须有两个轴不一样，but {mesh_idx_pair}"
    a_idx, a_d = mesh_idx_pair[0]
    b_idx, b_d = mesh_idx_pair[1]

    if (a_idx, b_idx) == (0, 1):
        three_axis_is = "z"
    elif (a_idx, b_idx) == (0, 2):
        three_axis_is = "y"
    elif (a_idx, b_idx) == (1, 2):
        three_axis_is = "x"
    else:
        raise Exception("不支持的轴")

    for x_step in range(points):
        for y_step in range(points):
            p = []
            for idx in range(6):
                if idx == a_idx:
                    p.append(corner1[idx] - a_d * x_step / (points - 1))
                elif idx == b_idx:
                    p.append(corner1[idx] - b_d * y_step / (points - 1))
                else:
                    p.append(corner1[idx])
            mesh_pose.append(np.array(p))

    return three_axis_is, mesh_pose

In [153]:
corner1h = np.array([215.65, 500.71, 50.01,  1.2106, 1.5345, 2.7614])  
corner2h = np.array([215.65, 459.93, 40.82,  1.2099, 1.5345, 2.7608]) 

center0h = np.array([215.65, 481.29, 46.84,  1.2092, 1.5345, 2.7601])  #Joints [45.624, -62.306, 116.356, -55.059, -133.224, -87.364]

points = 3
three_axis = 40
rotation = np.radians(16)
tool = [0.0, 0.01, 0.152]

three_axis_is, mesh_pose = extend_pose(corner1h, corner2h, points, three_axis, rotation, tool)

In [154]:
world.hStartRecord(r"D:\records\h_cam", 2000)

In [155]:
sleep_time = 0.8

all_pose = []

def shot():
    global all_pose
    world.hTrigger(len(all_pose), count=3)
    all_pose.append(world.getPose())
    time.sleep(0.6)

for idx, p in enumerate(mesh_pose):
    world.movePose(p)
    shot()

    world.movePoseDelta(y=three_axis)
    shot()

    world.movePoseDelta(y=-three_axis * 2)
    shot()

    world.movePose(p)
    if idx % 3 == 0:
        world.movePoseDelta(rx=rotation, tool=tool)
        shot()

        world.movePoseDelta(rx=-rotation *2, tool=tool)
        shot()

    elif idx % 3 == 1:
        world.movePoseDelta(ry=rotation, tool=tool)
        shot()

        world.movePoseDelta(ry=-rotation *2, tool=tool)
        shot()

    else:
        world.movePoseDelta(rz=rotation, tool=tool)
        shot()

        world.movePoseDelta(rz=-rotation *2, tool=tool)
        shot()

In [156]:
world.hStopRecord()

In [157]:
np.save('h_all_pose.npy',np.array([p.homo for p in all_pose]))

# 垂直相机

In [158]:
corner1v = np.array([178.18, 497.05, 34.47,  1.5667, -0.0207, 3.1114]) 
corner2v = np.array([228.75, 480.08, 34.48,  1.5667, -0.0207, 3.1114])  

center0v = np.array([204.27, 489.62, 34.48,  1.5667, -0.0207, 3.1114]) #Joints [47.573, -60.604, 116.056, -55.761, -130.69, 1.388]

points = 3
three_axis = 40
rotation = np.radians(16)
tool = [0.0, 0.01, 0.152]

three_axis_is, mesh_pose = extend_pose(corner1v, corner2v, points, three_axis, rotation, tool)

In [159]:
world.vStartRecord(r"D:\records\v_cam", 2000)

In [160]:
sleep_time = 0.8

all_pose = []

def shot():
    global all_pose
    world.vTrigger(len(all_pose), count=3)
    all_pose.append(world.getPose())
    time.sleep(0.6)

for idx, p in enumerate(mesh_pose):
    world.movePose(p)
    shot()

    world.movePoseDelta(y=three_axis)
    shot()

    world.movePoseDelta(y=-three_axis * 2)
    shot()

    world.movePose(p)
    if idx % 3 == 0:
        world.movePoseDelta(rx=rotation, tool=tool)
        shot()

        world.movePoseDelta(rx=-rotation *2, tool=tool)
        shot()
    elif idx % 3 == 1:
        world.movePoseDelta(ry=rotation, tool=tool)
        shot()

        world.movePoseDelta(ry=-rotation *2, tool=tool)
        shot()
    else:
        world.movePoseDelta(rz=rotation, tool=tool)
        shot()

        world.movePoseDelta(rz=-rotation *2, tool=tool)
        shot()

In [161]:
world.vStopRecord()

In [162]:
np.save('v_all_pose.npy',np.array([p.homo for p in all_pose]))